In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from sklearn import datasets

SEED=200
plt.ion()

# Implementar uma das duas funções abaixo:
## Reescalar entre 0 e 1 ou Normalização z-score

In [ ]:
def rescale(df, columns, maximum=None, minimum=None):
    """
    df: dataframe com o dataset
    columns: nomes das colunas que devem ser reescaladas
    maximum: dicionário com os valores maximos, com cada chave representando uma coluna
    minimum: dicionário com os valores minimos, com cada chave representando uma coluna
    retorna o dataset reescalado
    """
    raise NotImplementedError("implementar a função rescale")

def normalize(df, columns, mean=None, std=None):
    """
    df: dataframe com o dataset
    columns: nomes das colunas que devem ser normalizadas
    mean: dicionário com os valores médios, com cada chave representando uma coluna
    std: dicionário com os desvios padrão, com cada chave representando uma coluna
    retorna o dataset normalizado
    """
    raise NotImplementedError("implementar a função normalize")
    

# A célula seguinte carrega o dataset. Não é necessário modificar.

In [ ]:
"""Load dataset"""
iris = datasets.load_iris()

df_full = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                       columns= iris['feature_names'] + ['target'])

def get_train_test_inds(y, train_proportion=0.7):
    """
    y: coluna do atributo alvo
    retorna os indices de treino e teste estratificados pela classe
    """
    y=np.array(y)
    train_inds = np.zeros(len(y),dtype=bool)
    test_inds = np.zeros(len(y),dtype=bool)
    values = np.unique(y)
    for value in values:
        value_inds = np.nonzero(y==value)[0]
        np.random.shuffle(value_inds)
        n = int(train_proportion*len(value_inds))

        train_inds[value_inds[:n]]=True
        test_inds[value_inds[n:]]=True

    return train_inds,test_inds


train_inds, test_inds = get_train_test_inds(df_full.loc[:, "target"])
df_full[['target']] = df_full[['target']].astype(int)
df = df_full[train_inds]
df_val = df_full[test_inds]

In [ ]:
"""Use este trecho no caso de optar por reescalar"""
df, maximum, minimum = rescale(df, df.columns[0:4])
df_val, _, _ = rescale(df_val, df_val.columns[0:4], maximum, minimum)

"""Use este trecho no caso de optar por normalizar"""
# df, mean, std = normalize(df, df.columns[0:4])
# df_val, _, _ = normalize(df_val, df_val.columns[0:4], mean, std)

# Funções de Ativação e Softmax
## Implementar Relu, Sigmoid e Softmax

In [ ]:
class Relu(object):
    def forward(self, x):
        """
        x: Entrada (batch_size, numero_de_neuronios)
        retorna a saída da ativação relu (batch_size, numero de neuronios)
        """
        raise NotImplementedError("implementar o relu")
    
    def backward(self, x):
        """
        x: Entrada no backward (batch_size, numero_de_neuronios)
        retorna a saída da derivada da ativação relu (batch_size, numero_de_neuronios)
        """
        raise NotImplementedError("implementar o relu")

        
class Sigmoid(object):
    def forward(self, x):
        """
        x: Entrada (batch_size, numero_de_neuronios)
        retorna a saída da ativação sigmoidal (batch_size, numero_de_neuronios)
        """
        raise NotImplementedError("implementar o sigmoid")
        
    def backward(self, x):
        """
        x: Entrada no backward (batch_size, numero_de_neuronios)
        retorna a saída da derivada da ativação sigmoidal (batch_size, numero_de_neuronios)
        """
        raise NotImplementedError("implementar o sigmoid")
        

def softmax(x):
    """
    x: Entrada (batch_size, numero_de_neuronios)
    retorna a probabilidade de cada classe (batch_size, numero_de_neuronios)
    """
    raise NotImplementedError("implementar o softmax")
    


# A célula seguinte implementa uma Rede Neural com 2 camadas escondidas. 

In [ ]:
class NeuralNetwork(object):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size,
                 activation_fn='relu', learning_rate=1e-2, sigma=1., weight_decay=0.1):
        """Inicializacao pronta. Nao alterar"""
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.sigma = sigma
        
        # relu or sigmoid
        if activation_fn == 'relu':
            self.activation_fn = Relu()
        elif activation_fn == 'sigmoid':
            self.activation_fn = Sigmoid()
        
        # inicializa as matriz dos parametros
        self.params = {}
        self.params["w1"] = sigma * np.random.randn(input_size, hidden_size1)
        self.params["b1"] = np.zeros(hidden_size1)
        self.params["w2"] = sigma * np.random.randn(hidden_size1, hidden_size2)
        self.params["b2"] = np.zeros(hidden_size2)
        self.params["w3"] = sigma * np.random.randn(hidden_size2, output_size)
        self.params["b3"] = np.zeros(output_size)
        
        # inicializa as matrizes de gradientes
        self.grads = {}
        self.grads["w1"] = np.zeros((input_size, hidden_size1))
        self.grads["b1"] = np.zeros(hidden_size1)
        self.grads["w2"] = np.zeros((hidden_size1, hidden_size2))
        self.grads["b2"] = np.zeros(hidden_size2)
        self.grads["w3"] = np.zeros((hidden_size2, output_size))
        self.grads["b3"] = np.zeros(output_size)
        
    def forward(self, x):
        """
        x: entrada da rede neural
        retorna as probabilidades de cada classe (batch_size, numero_de_classes)
        """
        raise NotImplementedError("implementar o forward")
        
    def backward(self, y):
        """
        y: indices das classes esperadas (batch_size, 1)
        retorna o loss e o dicionario de gradientes
        """
        raise NotImplementedError("implementar o backward")        
        
    def optimize(self):
        """
        Faz um step do gradiente descendente, nao alterar
        """
        self.params["w3"] = self.params["w3"] - self.learning_rate*(self.grads['w3'])
        self.params["b3"] = self.params["b3"] - self.learning_rate*(self.grads['b3'])

        self.params["w2"] = self.params["w2"] - self.learning_rate*(self.grads['w2'])
        self.params["b2"] = self.params["b2"] - self.learning_rate*(self.grads['b2'])
        
        self.params["w1"] = self.params["w1"] - self.learning_rate*(self.grads['w1'])
        self.params["b1"] = self.params["b1"] - self.learning_rate*(self.grads['b1'])

# Devem ser alterados os hiperparâmetros para melhor otimização da rede
## Alguns dos hiperparâmetros que podem ser modificados estão na célula abaixo

In [ ]:
learning_rate = 1e-3
activation_fn = 'relu'
weight_decay = 0.001 # lambda
epochs = 1000
batch_size = 10

In [ ]:
classes = df.loc[:,"target"].unique() # numero de classes do dataset
classes.sort()

network = NeuralNetwork(df.shape[1]-1, 10, 10, output_size=len(classes), learning_rate=learning_rate,
                        activation_fn=activation_fn, weight_decay=weight_decay)

In [ ]:
"""Nao alterar esta celula"""
# Cria o grafico e atualiza os pesos
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(8,3))
ax1.set_title('Training Loss')
ax2.set_title('Training Accuracy')
ax3.set_title('Validation Accuracy')

t = tqdm(range(epochs))
losses = []
training_accuracy = []
accuracy = []

# Visualizacao do treinamento
drawn, = ax1.plot(np.arange(0), losses, c='r')
drawn2, = ax2.plot(np.arange(0), training_accuracy, c='r')
drawn3, = ax3.plot(np.arange(0), accuracy, c='r')

# Loop de treinamento
for e in t:
    # aleatoriza o treino e zera estatisticas
    df = df.sample(frac=1., random_state=SEED)
    total_loss = 0
    correct = 0
    total = 0
    # Loop da epoca
    for i in range(0, df.shape[0]-batch_size, batch_size):
        total += batch_size
        batch = df.iloc[i:(i+batch_size), :]
        y = df.iloc[i:(i+batch_size)]
        y = y.loc[:, "target"]
        batch = batch.drop(["target"], axis=1)
        predictions = network.forward(batch.values)
        loss, grads = network.backward(y)
        total_loss += loss
        network.optimize()
        correct += sum(y.values == predictions.argmax(axis=1))
    
    # Atualiza estatisticas e graficos
    training_accuracy.append(correct/total)
    losses.append(total_loss)
    t.set_description('Loss: %.3f' % total_loss)
    drawn.set_data((np.arange(len(losses)), losses))
    ax1.relim()
    ax1.set_xlim((0, e))
    ax1.autoscale_view()
    
    drawn2.set_data((np.arange(len(training_accuracy)), training_accuracy))
    ax2.relim()
    ax2.set_xlim((0, e))
    ax2.autoscale_view()
    
    ### VALIDACAO ###
    batch = df_val
    y = df_val.loc[:, "target"]
    batch = batch.drop(["target"], axis=1)

    predictions = network.forward(batch.values)
    correct = np.sum(y.values == predictions.argmax(axis=1))
    accuracy.append(correct/df_val.shape[0])
    
    # Atualiza grafico de validacao
    drawn3.set_data((np.arange(len(accuracy)), accuracy))
    ax3.relim()
    ax3.set_xlim((0, e))
    ax3.autoscale_view()
    fig.canvas.draw()